In [1]:
# Import the necessary libraries.

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")

# Display all the columns in a dataset

pd.set_option("display.max_columns", None)

import catboost
import sklearn

In [2]:
power = pd.read_csv(r"C:\Users\Anil\Desktop\data_science\53 Session 05-Aug-2021-20210805\Power_Plant_S1.csv",header = 0)
power.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [3]:
power.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [4]:
indepvar=[]
for col in power.columns:
    if col != "PE":
        indepvar.append(col)
targetvar = "PE"
x = power[indepvar]
y = power[targetvar]

In [5]:
# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=6)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((7654, 4), (1914, 4), (7654,), (1914,))

In [6]:
# Train the algorithm and build the model with train dataset

from catboost import CatBoostRegressor
from catboost.utils import eval_metric

modelCBR = CatBoostRegressor(iterations=100, verbose=10, loss_function='RMSE')
modelCBR.fit(x_train, y_train, eval_set=(x_test, y_test))
modelCBR.set_feature_names(x)

Learning rate set to 0.284146
0:	learn: 12.9377603	test: 12.9950235	best: 12.9950235 (0)	total: 147ms	remaining: 14.5s
10:	learn: 4.2528556	test: 4.2003767	best: 4.2003767 (10)	total: 183ms	remaining: 1.48s
20:	learn: 3.9803071	test: 3.9693375	best: 3.9693375 (20)	total: 209ms	remaining: 788ms
30:	learn: 3.8247720	test: 3.8803240	best: 3.8803240 (30)	total: 235ms	remaining: 523ms
40:	learn: 3.6822742	test: 3.7925573	best: 3.7911951 (39)	total: 260ms	remaining: 374ms
50:	learn: 3.5491183	test: 3.7104875	best: 3.7104875 (50)	total: 284ms	remaining: 273ms
60:	learn: 3.4571088	test: 3.6615576	best: 3.6615576 (60)	total: 308ms	remaining: 197ms
70:	learn: 3.3727371	test: 3.6230057	best: 3.6230057 (70)	total: 332ms	remaining: 136ms
80:	learn: 3.2923982	test: 3.5877199	best: 3.5877199 (80)	total: 357ms	remaining: 83.7ms
90:	learn: 3.2138860	test: 3.5452903	best: 3.5452903 (90)	total: 382ms	remaining: 37.8ms
99:	learn: 3.1653086	test: 3.5184673	best: 3.5184673 (99)	total: 406ms	remaining: 0us



In [7]:
# Predict the model

y_pred = modelCBR.predict(x_test)

# Evaluation metrics for CatBoost Regressor

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred), 3) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))

# Calculate Adjusted R squared values

r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 2.566
Mean Squared Error (MSE): 12.38
Root Mean Squared Error (RMSE): 3.518
Mean Absolute Percentage Error (MAPE): 0.6 %
R2_score: 0.957719
Adj R Square:  0.957701


In [8]:
# Important Attributes & Methods of CatBoostRegressor

print("Best Score : ", modelCBR.best_score_)
#print("\nList of Target Classses : ", modelCBR.classes_)
print("\nData Feature Names : ", modelCBR.feature_names_)
print("\nFeature Importance : ", modelCBR.feature_importances_)
print("\nLearning Rate : ", modelCBR.learning_rate_)
print("\nRandom Seed : ", modelCBR.random_seed_)
print("\nNumber of Trees : ", modelCBR.tree_count_)
print("\nNumber of Features : ", modelCBR.n_features_in_)

Best Score :  {'learn': {'RMSE': 3.1653086323852984}, 'validation': {'RMSE': 3.5184673175201495}}

Data Feature Names :  ['AT', 'V', 'AP', 'RH']

Feature Importance :  [62.20962503 25.46271393  8.0984995   4.22916154]

Learning Rate :  0.2841460108757019

Random Seed :  0

Number of Trees :  100

Number of Features :  4


In [10]:
# CatBoost Methods:

print("\nBest Score : ", modelCBR.get_best_score())
print("\nParameters Passed When Creating Model : ", modelCBR.get_params())
print("\nAll Model Parameters : ", modelCBR.get_all_params())
#print("\nCategorical Feature Indices : ", modelCBR.get_cat_feature_indices())
print("\nFeature Importances : ", modelCBR.get_feature_importance())
print("\nLeaf Values Shape : ", modelCBR.get_leaf_values().shape)
print("\nLeaf Values : ", modelCBR.get_leaf_values()[:10])
print("\nLeaft Weights Shape : ", modelCBR.get_leaf_weights().shape)
print("\nLeaft Weights : ", modelCBR.get_leaf_weights()[:10])


Best Score :  {'learn': {'RMSE': 3.1653086323852984}, 'validation': {'RMSE': 3.5184673175201495}}

Parameters Passed When Creating Model :  {'iterations': 100, 'loss_function': 'RMSE', 'verbose': 10}

All Model Parameters :  {'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 100, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 0.800000011920929, 'use_best_model': True, 'random_seed': 0, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1

In [11]:
# Build the CatBoost with Pool function => discuss

from catboost import Pool

# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

train_data = catboost.Pool(x_train, y_train)
test_data = catboost.Pool(x_test, y_test)

# Build the model

from catboost import CatBoostRegressor
from catboost import CatBoost
from catboost.utils import eval_metric

modelCBR = catboost.CatBoostRegressor(loss_function='RMSE')

params = {'iterations': [100, 150, 200],
'learning_rate': [0.03, 0.1],
'depth': [2, 4, 6, 8],
'l2_leaf_reg': [0.2, 0.5, 1, 3]}

modelCBR.grid_search(params, train_data)

y_pred = modelCBR.predict(test_data)

# Evaluation metrics for CatBoost Regressor

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred), 3) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))

# Calculate Adjusted R squared values

r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

0:	learn: 441.0567038	test: 441.3533869	best: 441.3533869 (0)	total: 2.26ms	remaining: 224ms
1:	learn: 427.8442074	test: 428.1426105	best: 428.1426105 (1)	total: 4.52ms	remaining: 222ms
2:	learn: 415.0156087	test: 415.3135560	best: 415.3135560 (2)	total: 6.49ms	remaining: 210ms
3:	learn: 402.5833472	test: 402.8717144	best: 402.8717144 (3)	total: 8.37ms	remaining: 201ms
4:	learn: 390.5155435	test: 390.8045152	best: 390.8045152 (4)	total: 9.88ms	remaining: 188ms
5:	learn: 378.8067946	test: 379.0900322	best: 379.0900322 (5)	total: 11.2ms	remaining: 175ms
6:	learn: 367.4553101	test: 367.7372578	best: 367.7372578 (6)	total: 12.5ms	remaining: 167ms
7:	learn: 356.4440801	test: 356.7292176	best: 356.7292176 (7)	total: 14.3ms	remaining: 165ms
8:	learn: 345.7571679	test: 346.0301457	best: 346.0301457 (8)	total: 15.6ms	remaining: 157ms
9:	learn: 335.3907848	test: 335.6441164	best: 335.6441164 (9)	total: 16.7ms	remaining: 150ms
10:	learn: 325.3448531	test: 325.5999672	best: 325.5999672 (10)	total:

55:	learn: 4.7878178	test: 4.7509531	best: 4.7509531 (55)	total: 62.4ms	remaining: 49ms
56:	learn: 4.7430033	test: 4.7106437	best: 4.7106437 (56)	total: 63.5ms	remaining: 47.9ms
57:	learn: 4.7078390	test: 4.6782520	best: 4.6782520 (57)	total: 64.7ms	remaining: 46.9ms
58:	learn: 4.6700210	test: 4.6450198	best: 4.6450198 (58)	total: 66ms	remaining: 45.8ms
59:	learn: 4.6448818	test: 4.6267250	best: 4.6267250 (59)	total: 67.3ms	remaining: 44.8ms
60:	learn: 4.6208142	test: 4.6070744	best: 4.6070744 (60)	total: 68.4ms	remaining: 43.7ms
61:	learn: 4.5976002	test: 4.5862990	best: 4.5862990 (61)	total: 69.5ms	remaining: 42.6ms
62:	learn: 4.5782878	test: 4.5671783	best: 4.5671783 (62)	total: 70.5ms	remaining: 41.4ms
63:	learn: 4.5605023	test: 4.5516204	best: 4.5516204 (63)	total: 71.5ms	remaining: 40.2ms
64:	learn: 4.5455915	test: 4.5368556	best: 4.5368556 (64)	total: 72.6ms	remaining: 39.1ms
65:	learn: 4.5308251	test: 4.5238782	best: 4.5238782 (65)	total: 73.6ms	remaining: 37.9ms
66:	learn: 4.5

15:	learn: 84.6102330	test: 84.6690991	best: 84.6690991 (15)	total: 18ms	remaining: 94.6ms
16:	learn: 76.2099672	test: 76.2690747	best: 76.2690747 (16)	total: 19.2ms	remaining: 93.9ms
17:	learn: 68.6712821	test: 68.7255665	best: 68.7255665 (17)	total: 20.3ms	remaining: 92.4ms
18:	learn: 61.8714069	test: 61.9281112	best: 61.9281112 (18)	total: 21.3ms	remaining: 90.8ms
19:	learn: 55.7501144	test: 55.7963456	best: 55.7963456 (19)	total: 22.3ms	remaining: 89.2ms
20:	learn: 50.2738412	test: 50.3213752	best: 50.3213752 (20)	total: 23.3ms	remaining: 87.5ms
21:	learn: 45.3143790	test: 45.3356784	best: 45.3356784 (21)	total: 24.2ms	remaining: 85.8ms
22:	learn: 40.8612339	test: 40.8769113	best: 40.8769113 (22)	total: 25.3ms	remaining: 84.5ms
23:	learn: 36.8620909	test: 36.8658258	best: 36.8658258 (23)	total: 26.2ms	remaining: 83ms
24:	learn: 33.2714545	test: 33.2669594	best: 33.2669594 (24)	total: 27.2ms	remaining: 81.6ms
25:	learn: 30.0336333	test: 30.0137375	best: 30.0137375 (25)	total: 28.1ms

71:	learn: 51.3334872	test: 51.4247570	best: 51.4247570 (71)	total: 73.3ms	remaining: 28.5ms
72:	learn: 49.8163281	test: 49.9051223	best: 49.9051223 (72)	total: 74.4ms	remaining: 27.5ms
73:	learn: 48.3485604	test: 48.4357339	best: 48.4357339 (73)	total: 75.4ms	remaining: 26.5ms
74:	learn: 46.9313302	test: 47.0176217	best: 47.0176217 (74)	total: 76.4ms	remaining: 25.5ms
75:	learn: 45.5479976	test: 45.6298888	best: 45.6298888 (75)	total: 77.4ms	remaining: 24.5ms
76:	learn: 44.2067645	test: 44.2878063	best: 44.2878063 (76)	total: 78.6ms	remaining: 23.5ms
77:	learn: 42.9039941	test: 42.9800320	best: 42.9800320 (77)	total: 79.5ms	remaining: 22.4ms
78:	learn: 41.6415501	test: 41.7130222	best: 41.7130222 (78)	total: 80.5ms	remaining: 21.4ms
79:	learn: 40.4179199	test: 40.4869312	best: 40.4869312 (79)	total: 81.6ms	remaining: 20.4ms
80:	learn: 39.2403856	test: 39.3090941	best: 39.3090941 (80)	total: 82.6ms	remaining: 19.4ms
81:	learn: 38.0897355	test: 38.1531020	best: 38.1531020 (81)	total: 83

28:	learn: 188.4868919	test: 188.6616329	best: 188.6616329 (28)	total: 30.3ms	remaining: 74.1ms
29:	learn: 182.8518606	test: 183.0106695	best: 183.0106695 (29)	total: 31.3ms	remaining: 73.1ms
30:	learn: 177.3855644	test: 177.5439007	best: 177.5439007 (30)	total: 32.4ms	remaining: 72ms
31:	learn: 172.0922701	test: 172.2496568	best: 172.2496568 (31)	total: 33.5ms	remaining: 71.2ms
32:	learn: 166.9506464	test: 167.1090965	best: 167.1090965 (32)	total: 34.7ms	remaining: 70.5ms
33:	learn: 161.9584124	test: 162.1152261	best: 162.1152261 (33)	total: 35.7ms	remaining: 69.3ms
34:	learn: 157.1195100	test: 157.2787998	best: 157.2787998 (34)	total: 36.9ms	remaining: 68.5ms
35:	learn: 152.4332146	test: 152.5958985	best: 152.5958985 (35)	total: 38.2ms	remaining: 67.9ms
36:	learn: 147.8868885	test: 148.0449336	best: 148.0449336 (36)	total: 39.3ms	remaining: 66.9ms
37:	learn: 143.4671729	test: 143.6152343	best: 143.6152343 (37)	total: 40.4ms	remaining: 65.9ms
38:	learn: 139.1872439	test: 139.3361356	b

...

111:	learn: 16.8736703	test: 16.9170994	best: 16.9170994 (111)	total: 561ms	remaining: 440ms
112:	learn: 16.4276188	test: 16.4744750	best: 16.4744750 (112)	total: 565ms	remaining: 435ms
113:	learn: 15.9956001	test: 16.0427700	best: 16.0427700 (113)	total: 570ms	remaining: 430ms
114:	learn: 15.5766040	test: 15.6266183	best: 15.6266183 (114)	total: 575ms	remaining: 425ms
115:	learn: 15.1689322	test: 15.2225960	best: 15.2225960 (115)	total: 580ms	remaining: 420ms
116:	learn: 14.7789461	test: 14.8344026	best: 14.8344026 (116)	total: 585ms	remaining: 415ms
117:	learn: 14.3990922	test: 14.4570011	best: 14.4570011 (117)	total: 593ms	remaining: 412ms
118:	learn: 14.0338843	test: 14.0918309	best: 14.0918309 (118)	total: 598ms	remaining: 407ms
119:	learn: 13.6790747	test: 13.7399197	best: 13.7399197 (119)	total: 604ms	remaining: 402ms
120:	learn: 13.3331454	test: 13.3964508	best: 13.3964508 (120)	total: 608ms	remaining: 397ms
121:	learn: 13.0039690	test: 13.0677416	best: 13.0677416 (121)	total: 

21:	learn: 47.0917005	test: 47.0873509	best: 47.0873509 (21)	total: 104ms	remaining: 841ms
22:	learn: 42.5774182	test: 42.5696189	best: 42.5696189 (22)	total: 109ms	remaining: 836ms
23:	learn: 38.4836887	test: 38.4785350	best: 38.4785350 (23)	total: 113ms	remaining: 830ms
24:	learn: 34.8144311	test: 34.7978575	best: 34.7978575 (24)	total: 118ms	remaining: 825ms
25:	learn: 31.4915638	test: 31.4862575	best: 31.4862575 (25)	total: 122ms	remaining: 820ms
26:	learn: 28.5185470	test: 28.5131987	best: 28.5131987 (26)	total: 127ms	remaining: 814ms
27:	learn: 25.8563752	test: 25.8529854	best: 25.8529854 (27)	total: 132ms	remaining: 811ms
28:	learn: 23.4797747	test: 23.4802816	best: 23.4802816 (28)	total: 137ms	remaining: 806ms
29:	learn: 21.3298193	test: 21.3369906	best: 21.3369906 (29)	total: 142ms	remaining: 804ms
30:	learn: 19.4286427	test: 19.4472093	best: 19.4472093 (30)	total: 147ms	remaining: 800ms
31:	learn: 17.7112559	test: 17.7421541	best: 17.7421541 (31)	total: 151ms	remaining: 794ms

138:	learn: 4.0738348	test: 4.7427316	best: 4.7427316 (138)	total: 651ms	remaining: 286ms
139:	learn: 4.0655577	test: 4.7379624	best: 4.7379624 (139)	total: 656ms	remaining: 281ms
140:	learn: 4.0596942	test: 4.7320855	best: 4.7320855 (140)	total: 661ms	remaining: 276ms
141:	learn: 4.0535394	test: 4.7231394	best: 4.7231394 (141)	total: 666ms	remaining: 272ms
142:	learn: 4.0477390	test: 4.7196677	best: 4.7196677 (142)	total: 671ms	remaining: 267ms
143:	learn: 4.0397705	test: 4.7178913	best: 4.7178913 (143)	total: 676ms	remaining: 263ms
144:	learn: 4.0302337	test: 4.7122068	best: 4.7122068 (144)	total: 681ms	remaining: 258ms
145:	learn: 4.0248983	test: 4.7058070	best: 4.7058070 (145)	total: 687ms	remaining: 254ms
146:	learn: 4.0175868	test: 4.7025643	best: 4.7025643 (146)	total: 692ms	remaining: 250ms
147:	learn: 4.0115419	test: 4.6975399	best: 4.6975399 (147)	total: 698ms	remaining: 245ms
148:	learn: 4.0040536	test: 4.6940637	best: 4.6940637 (148)	total: 703ms	remaining: 241ms
149:	learn

62:	learn: 4.1221967	test: 4.3826962	best: 4.3826962 (62)
63:	learn: 4.1049444	test: 4.3664995	best: 4.3664995 (63)
64:	learn: 4.0850053	test: 4.3470002	best: 4.3470002 (64)
65:	learn: 4.0712946	test: 4.3339439	best: 4.3339439 (65)
66:	learn: 4.0546411	test: 4.3208016	best: 4.3208016 (66)
67:	learn: 4.0423296	test: 4.3114497	best: 4.3114497 (67)
68:	learn: 4.0317209	test: 4.3045117	best: 4.3045117 (68)
69:	learn: 4.0182318	test: 4.2938158	best: 4.2938158 (69)
70:	learn: 4.0087492	test: 4.2872117	best: 4.2872117 (70)
71:	learn: 3.9947438	test: 4.2749915	best: 4.2749915 (71)
72:	learn: 3.9822375	test: 4.2642262	best: 4.2642262 (72)
73:	learn: 3.9728487	test: 4.2557291	best: 4.2557291 (73)
74:	learn: 3.9622964	test: 4.2467123	best: 4.2467123 (74)
75:	learn: 3.9536315	test: 4.2405639	best: 4.2405639 (75)
76:	learn: 3.9447415	test: 4.2319145	best: 4.2319145 (76)
77:	learn: 3.9362009	test: 4.2250423	best: 4.2250423 (77)
78:	learn: 3.9231175	test: 4.2168125	best: 4.2168125 (78)
79:	learn: 3.9

Mean Absolute Error (MAE): 2.59
Mean Squared Error (MSE): 11.833
Root Mean Squared Error (RMSE): 3.44
Mean Absolute Percentage Error (MAPE): 0.6 %
R2_score: 0.959204
Adj R Square:  0.959187


In [13]:
params = modelCBR.get_params()
print(params)

{'loss_function': 'RMSE', 'depth': 6, 'iterations': 200, 'learning_rate': 0.1, 'l2_leaf_reg': 0.2}


###  multi regression

In [14]:
# Train the algorithm and build the model with train dataset

from sklearn.linear_model import LinearRegression

modelREG = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None, positive=False)

modelREG.fit(x_train, y_train)
# Predict the model

y_pred = modelREG.predict(x_test)

# Evaluation metrics for CatBoost Regressor

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred), 3) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))

# Calculate Adjusted R squared values

r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 3.596
Mean Squared Error (MSE): 20.274
Root Mean Squared Error (RMSE): 4.503
Mean Absolute Percentage Error (MAPE): 0.8 %
R2_score: 0.930105
Adj R Square:  0.930076


## decision tree

In [15]:
# Build the Decision Tree Regressor model

from sklearn.tree import DecisionTreeRegressor

modelDTR = DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=None, min_samples_split=2,
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None,
                                 random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                 min_impurity_split=None, ccp_alpha=0.0)

modelDTR.fit(x_train, y_train)
# Predict the model with test dataset

y_pred1 = modelDTR.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred1),3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred1),3))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred1)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred1),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred1),6))

# Calculate Adjusted R squared values

r_squared = round(metrics.r2_score(y_test, y_pred1),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 3.021
Mean Squared Error (MSE): 19.537
Root Mean Squared Error (RMSE): 4.42
Mean Absolute Percentage Error (MAPE): 0.67 %
R2_score: 0.932646
Adj R Square:  0.932618


## Random Forest Regressor

In [16]:
# build a random forest regression model with optimized hyper parameters

from sklearn.ensemble import RandomForestRegressor

bostonRFR = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2,
min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
bootstrap=True, oob_score=False, n_jobs=None, random_state=None,
verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)

bostonRFR.fit(x_train, y_train)

# Predict the model with test dataset

y_pred2 = bostonRFR.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred2),3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred2),3))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred2)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred2),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred2),6))

# Calculate Adjusted R squared values

r_squared = round(metrics.r2_score(y_test, y_pred2),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 2.324
Mean Squared Error (MSE): 10.446
Root Mean Squared Error (RMSE): 3.232
Mean Absolute Percentage Error (MAPE): 0.51 %
R2_score: 0.963986
Adj R Square:  0.963971


## svr linear kernel

In [17]:
# Build the SVR model

from sklearn.svm import SVR

bostonSVR = SVR(C=1.0, kernel='linear', degree=3, gamma='scale', coef0=0.0, tol=0.001,
epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=- 1)

bostonSVR.fit(x_train, y_train)

# Predict the model with test dataset

y_pred3 = bostonSVR.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred3),3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred3),3))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred3)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred3),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred3),6))

# Calculate Adjusted R squared values

r_squared = round(metrics.r2_score(y_test, y_pred3),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 3.598
Mean Squared Error (MSE): 20.587
Root Mean Squared Error (RMSE): 4.537
Mean Absolute Percentage Error (MAPE): 0.79 %
R2_score: 0.929025
Adj R Square:  0.928995


## knn regressor

In [18]:
# Build the algorithm with KNN

from sklearn import neighbors

bostonKNN = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto',
leaf_size=30, p=2, metric='minkowski',
metric_params=None, n_jobs=None)
bostonKNN.fit(x_train, y_train)

# Predict the model with test dataset

y_pred4 = bostonKNN.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred4),3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred4),3))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred4)),3))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred4),4) * 100, '%')
print('R2_score:', round(metrics.r2_score(y_test, y_pred4),6))

# Calculate Adjusted R squared values

r_squared = round(metrics.r2_score(y_test, y_pred4),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 2.884
Mean Squared Error (MSE): 15.383
Root Mean Squared Error (RMSE): 3.922
Mean Absolute Percentage Error (MAPE): 0.64 %
R2_score: 0.946965
Adj R Square:  0.946943
